In [50]:
# -*- coding: utf-8 -*-
%matplotlib inline
import re, jieba
import gensim
import difflib
import numpy as np
import random
import opencc
from smart_open import smart_open
import matplotlib.pyplot as plt
import subprocess

ANS = ['a', 'b', 'c', 'd', 'e']
WINDOW = 10
VEC_SIZE = 100
def simple_preprocess(content):
    content = content.strip().replace('︽⊙＿⊙︽', '龐燮傍謝')
    wlist = list(jieba.cut(content))
    qidx = []
    i = 0
    for w in wlist:
        if w == '龐燮傍謝':
            wlist[i] = '*'
            qidx.append(i)
        i += 1
    return (wlist, qidx)

def normalize_vec(vec):
    mag = ((vec * vec).sum()) ** 0.5
    return vec / mag

def build_estimate_samples(wlist, qidx):
    global WINDOW
    temp = wlist[:]
    est_sen = []
    sen_len = len(wlist)
    for i in qidx:
        head = max(i - WINDOW, 0)
        tail = min(i + WINDOW, sen_len)
        est_sen.append(wlist[head : i] + wlist[i + 1 : tail])
    return est_sen

def estimate_ans(est_sen_list, options):
    global VEC_SIZE
    n_sen = float(len(est_sen_list))
    option_vec_idx = []
    for w in options:
        if w in model: option_vec_idx.append(model.vocab[w].index)
        else: 
            option_vec_idx.append(-1)
            return -1
    score = [0., 0., 0., 0., 0.]
    for wlist in est_sen_list:
        arr = np.zeros(VEC_SIZE)
        for w in wlist:
            if w in model and w != u'*': arr += model[w]
        for i in range(5):
#             if option_vec_idx[i] >= 0: score[i] += np.dot(normalize_vec(arr), 
#                                                           normalize_vec(model[model.index2word[option_vec_idx[i]]]))
            if option_vec_idx[i] >= 0: score[i] += np.dot(arr, 
                                                          syn1neg[option_vec_idx[i]])
    
    for i in range(5):
        score[i] /= n_sen
    
    return ANS[score.index(max(score))]
#     np.dot(arr, syn1neg[idx])



In [46]:
# jieba.add_word('龐燮傍謝', freq=10, tag='xx')
# jieba.load_userdict('data/zhwiki-cn-clean')
# jieba.load_userdict('data/dict-txt-big')

path = 'w2v-experiment/model/'
model = gensim.models.Word2Vec.load_word2vec_format(path + 'sk-syn0.bin', binary = True)
vocab_size, vector_size = model.syn0.shape
syn1neg = np.zeros((vocab_size, vector_size), dtype=np.float32)
binary_len = np.dtype(np.float32).itemsize * vector_size
with smart_open(path + 'sk-syn1neg.bin') as fin:
    for i in range(vocab_size):
        weights = np.fromstring(fin.read(binary_len), dtype=np.float32)
        syn1neg[i] = weights

In [52]:
right, wrong = 0, 0
wrong_sen = []
with open('question_official/hackathon_1000_cn.tsv', 'rb') as f:
    for i in range(200): next(f)
    for line in f:
#         s = opencc.convert(line.decode('utf-8'), config = '/usr/share/opencc/tw2s.json')
        s = line.decode('utf-8')
#         print(repr(s))
        parse = s.split('\t')
#         print(parse)
        no = int(parse[0])
        content = parse[1].strip()
        ans = parse[2]
        options = parse[3 : 8]
        ans_ref = parse[8]
        url = parse[9]
        level = parse[10]
#         print(content)
#         print(options, ans_ref)
        wlist, qidx = simple_preprocess(content)
        est_sen = build_estimate_samples(wlist, qidx)
        pred = estimate_ans(est_sen, options)
        if pred == -1:
#             wrong += 1
            continue
        if pred == ans: right += 1
        else: 
            wrong += 1
            wrong_sen.append((content, options[ANS.index(pred)], ans_ref))
#         break
print(float(right)/float(right+wrong))
#  (['脸上', '比较', '油', '的', '时候', '我', '都', '会', '习惯', '让', '*', '多', '停留', '个', '10', 
#    '秒左右', '，', '会', '比', '马上', '就', '洗掉', '清洁', '得', '更', '干净', '一点', '唷'], [10])
# v100w10n50p10i1 : 0.764065335753176
# v100w10n50p10i10 : 0.7985480943738656
# v100w10n50p10i30 : 0.8076225045372051
# v100w20n50p10i10 : 0.79491833030853
# v80w5n50p10i30 : 0.7531760435571688
# v200w10n50p10i10 : 0.8148820326678766
# v200w10n50p10i50 : 0.7912885662431942
# v200w10n50p10i30 : 0.7586206896551724
# v100w10n50p10i10hs1 : 0.7967332123411979
# v100w10n50p1i10 : 0.79491833030853

# cbow: 0.8137404580152672
# skgram: 0.7633587786259542

0.7633587786259542


In [11]:
qidx

[]

In [12]:
model.most_similar(['松烟文创'])

[('松山文创', 0.7016356587409973),
 ('南港软体', 0.6479178667068481),
 ('旁Wiz', 0.6130987405776978),
 ('Waffogato', 0.5951836705207825),
 ('@内科', 0.5815931558609009),
 ('内湖', 0.5726386904716492),
 ('南港', 0.5714350938796997),
 ('@捷运港墘站', 0.5695842504501343),
 ('榆小', 0.5559353828430176),
 ('松烟', 0.5554336309432983)]

In [43]:
word_dict = ' '.join(list(model.vocab))

In [44]:
for match in re.findall(r'\s(\S*文创\S*)\s', word_dict): 
    print(match)
    

台艺大文创
文创区里
青青文创
花莲文创产业
台北文创
文创产业
文创
十鼓文创
著文创
猫头鹰文创
文创生活
成文创
起初文创
经贸文创
灿星文创
华山文创
学学文创
台北文创大楼
图写文创
文创头围
文创聚落
谷物文创
头围文创
文创工作者
梦田文创
松烟文创
美地文创
拥恒文创
莎贝莉娜文创
文创产业园区
丰味文创
甘乐文创
松山文创
文创化
华山文创产业
文创园区
文创舘


In [38]:
# model[model.index2word[1]]
vocab_size

975332